# Model Training

We will first load the data and scale it

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df_train = pd.read_csv('./dataset/train.csv')
df_val = pd.read_csv('./dataset/val.csv')
df_test = pd.read_csv('./dataset/test.csv')

scaler_r = MinMaxScaler()
scaler_c = MinMaxScaler()

scaler_r.fit(df_test[['registered']])
scaler_c.fit(df_test[['casual']])


df_train['registered_s'] = scaler_r.transform(df_train[['registered']])
df_val['registered_s'] = scaler_r.transform(df_val[['registered']])

df_train['casual_s'] = scaler_c.transform(df_train[['casual']])
df_val['casual_s'] = scaler_c.transform(df_val[['casual']])

In [2]:
df_train.head()

,Unnamed: 0,ds,season,workingday,weathersit,temp,hum,casual,registered,unique_id,day_sin,day_cos,week_sin,week_cos,registered_s,casual_s
0,0,2011-01-01 00:00:00,1,0,1,0.24,0.81,3,13,bike_sharing,0.000000,1.000000,-0.781831,0.62349,0.014840,0.008646
1,1,2011-01-01 01:00:00,1,0,1,0.22,0.80,8,32,bike_sharing,0.258819,0.965926,-0.781831,0.62349,0.036530,0.023055
2,2,2011-01-01 02:00:00,1,0,1,0.22,0.80,5,27,bike_sharing,0.500000,0.866025,-0.781831,0.62349,0.030822,0.014409
3,3,2011-01-01 03:00:00,1,0,1,0.24,0.75,3,10,bike_sharing,0.707107,0.707107,-0.781831,0.62349,0.011416,0.008646
4,4,2011-01-01 04:00:00,1,0,1,0.24,0.75,0,1,bike_sharing,0.866025,0.500000,-0.781831,0.62349,0.001142,0.000000


In [3]:
df_train.drop(['Unnamed: 0','casual','registered'],axis=1,inplace=True)
df_val.drop(['Unnamed: 0','casual','registered'],axis=1,inplace=True)

df_train.head()

,ds,season,workingday,weathersit,temp,hum,unique_id,day_sin,day_cos,week_sin,week_cos,registered_s,casual_s
0,2011-01-01 00:00:00,1,0,1,0.24,0.81,bike_sharing,0.000000,1.000000,-0.781831,0.62349,0.014840,0.008646
1,2011-01-01 01:00:00,1,0,1,0.22,0.80,bike_sharing,0.258819,0.965926,-0.781831,0.62349,0.036530,0.023055
2,2011-01-01 02:00:00,1,0,1,0.22,0.80,bike_sharing,0.500000,0.866025,-0.781831,0.62349,0.030822,0.014409
3,2011-01-01 03:00:00,1,0,1,0.24,0.75,bike_sharing,0.707107,0.707107,-0.781831,0.62349,0.011416,0.008646
4,2011-01-01 04:00:00,1,0,1,0.24,0.75,bike_sharing,0.866025,0.500000,-0.781831,0.62349,0.001142,0.000000


Now Lets create the dataset from the tabular data to work with our model

In [4]:
import numpy as np


def create_dataset(data, feature_columns, target_columns, lookback_window, horizon, shift):
    """
    Transforms a time series DataFrame into input-target pairs for a deep learning model.

    Args:
        data (pd.DataFrame): The input DataFrame.
        feature_columns (list): List of column names to use as input features (X).
        target_columns (list): List of column names to use as target variables (Y).
        lookback_window (int): The number of past time steps to use as input (X).
        horizon (int): The number of future time steps to predict (Y).
        shift (int): The number of steps to shift the window for each new sample.

    Returns:
        tuple: A tuple containing two NumPy arrays, X (inputs) and Y (targets).
    """
    X, Y = [], []
    for i in range(0, len(data) - lookback_window - horizon + 1, shift):
        # Extract the input features (X) for the current window
        x_start = i
        x_end = i + lookback_window
        X.append(data.iloc[x_start:x_end][feature_columns].values)

        # Extract the target variables (Y) for the forecast horizon
        y_start = i + lookback_window
        y_end = y_start + horizon
        Y.append(data.iloc[y_start:y_end][target_columns].values)
    
    return np.array(X), np.array(Y)


I am using a yaml file to load some parameters as I will need to access these across several notebooks

In [5]:
import yaml

def load_config(filepath):
    """
    Loads a YAML configuration file from the specified filepath.

    Args:
        filepath (str): The path to the YAML file.

    Returns:
        dict: The configuration as a dictionary.
    """
    try:
        with open(filepath, 'r') as file:
            config = yaml.safe_load(file)
        return config
    except FileNotFoundError:
        print(f"Error: The file at {filepath} was not found.")
        return None
    except yaml.YAMLError as exc:
        print(f"Error parsing YAML file: {exc}")
        return None

# Load the configuration from the YAML file
config_file = 'config.yaml'
model_config = load_config(config_file)

In [6]:
#  Define key parameters
LOOKBACK_WINDOW = model_config['dataset']['lookback_window']  
FORECAST_HORIZON = model_config['dataset']['forecast_horizon'] 
SHIFT = model_config['dataset']['shift']          
# ----------------------------------------------------
# The columns model use to learn the patterns.

feature_columns = model_config['data']['feature_columns']

# The columns model will try to predict.
target_columns = model_config['data']['target_columns']



Lets' first create the training dataset and have a look at it

In [7]:
X_train, Y_train = create_dataset(df_train, feature_columns, target_columns, LOOKBACK_WINDOW, FORECAST_HORIZON, SHIFT)
X_val, Y_val = create_dataset(df_val, feature_columns, target_columns, LOOKBACK_WINDOW, FORECAST_HORIZON, SHIFT)


print(f"Shape of Input (X): {X_train.shape}")       # shape will be in the following order
print(f"Shape of Target (Y): {Y_train.shape}")      # [Number of Samples, Window, Number of Features]


Shape of Input (X): (13735, 168, 7)
Shape of Target (Y): (13735, 1, 2)


Now Let's create dataloader which will serve the above created data to our models for training and testing.

In [8]:
import torch
from torch.utils.data import DataLoader, TensorDataset

BATCH_SIZE = model_config['dataset']['batch_size']

# Convert NumPy arrays to PyTorch Tensors
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(Y_train)
X_val = torch.Tensor(X_val)
y_val = torch.Tensor(Y_val)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


## Designing the model

Now let's Design our model.  
We will create two models. one solely based on LSTM architecture and another one combining CNN with LSTM.
I have saved this same script on model.py so I can access these models across the notebooks.


In [ ]:
import torch.nn as nn

class LSTMForecaster(nn.Module):
    def __init__(self, num_features, hidden_size, num_layers, output_size):
        super(LSTMForecaster,self).__init__()

        self.num_features = num_features
        self.hidden_size = hidden_size
        self.num_layer = num_layers
        self.output_size = output_size

        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self,x):
        lstm_out,(h_n,c_n) = self.lstm(x)
        out = lstm_out[:,-1,:]  # Getting the output for the final step.
        out = self.fc(out)
        out = out.view(x.size(0),-1,2) #changing the shape of the output to (batch_size,horizen,targets)
        return out
    

class CNN_LSTMForecaster(nn.Module):
    def __init__(self, num_features, hidden_size, num_layers, output_size, cnn_filters=64, kernel_size=7):
        super(CNN_LSTMForecaster, self).__init__()
        
        self.num_features = num_features
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.cnn_filters = cnn_filters
        self.kernel_size = kernel_size

        self.cnn = nn.Sequential(
            # Input shape: (batch_size, num_features, lookback_window)
            nn.Conv1d(
                in_channels=num_features, 
                out_channels=self.cnn_filters, 
                kernel_size=self.kernel_size,
                padding='same'
            ),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )

        self.lstm = nn.LSTM(
            input_size=self.cnn_filters,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            batch_first=True
        )

        self.fc = nn.Linear(self.hidden_size, self.output_size)

    def forward(self,x):

        #shape of x is (batch_size, lookback_window, num_features)
        x = x.permute(0, 2, 1) # for the convolution we need shape of x to be (batch_size, num_features, lookback_window)
        cnn_out = self.cnn(x)
        cnn_out = cnn_out.permute(0, 2, 1) #changing the shape back for LSTM
        lstm_out, (h_n, c_n) = self.lstm(cnn_out)
        out = lstm_out[:,-1,:]  # Getting the output for the final step.
        out = self.fc(out)
        out = out.view(x.size(0), -1, 2)
        
        return out

Let's initialize the both models.

In [10]:
import torch
import torch.nn as nn

BATCH_SIZE = model_config['dataset']['batch_size']
LOOKBACK_WINDOW = model_config['dataset']['lookback_window']  
NUM_FEATURES = model_config['model']['num_features']
FORECAST_HORIZON = model_config['dataset']['forecast_horizon'] 
NUM_TARGETS = model_config['model']['num_targets']


device = 'cuda' if torch.cuda.is_available() else 'cpu'

lstm_model = LSTMForecaster(
        num_features=NUM_FEATURES,
        hidden_size=64,
        num_layers=1,
        output_size=FORECAST_HORIZON * NUM_TARGETS
    )

cnn_lstm_model = CNN_LSTMForecaster(
        num_features=NUM_FEATURES,
        hidden_size=64,
        num_layers=1,
        output_size=FORECAST_HORIZON * NUM_TARGETS
    )

with everything ready, let's train our models

In [11]:
from tqdm import tqdm

def train_model(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device='cpu', name=''):
    """
    Trains and validates a time series forecasting model.

    Args:
        model (nn.Module): The forecasting model to train.
        optimizer (torch.optim.Optimizer): The optimizer to use for training.
        loss_fn (nn.Module): The loss function (e.g., MSELoss).
        train_loader (DataLoader): DataLoader for the training data.
        val_loader (DataLoader): DataLoader for the validation data.
        epochs (int): The number of training epochs.
        device (str): The device to run the training on ('cpu' or 'cuda').
    """
    best_val_loss = float('inf')
    best_model_state = None

    model.to(device)

    for epoch in range(epochs):
        # --- Training Loop ---
        model.train()
        train_loss = 0.0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} Training"):
            optimizer.zero_grad() # Reset gradients
            
            inputs,targets = batch
            inputs=inputs.to(device)
            targets=targets.to(device)

            output = model(inputs)
            
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * targets.size(0)
        
        avg_train_loss = train_loss / len(train_loader.dataset)

        # --- Validation Loop ---
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} Validating"):
                optimizer.zero_grad() # Reset gradients
                
                inputs,targets = batch
                inputs=inputs.to(device)
                targets=targets.to(device)

                output = model(inputs)
                
                loss = loss_fn(output, targets)
                
                val_loss += loss.item() * targets.size(0)

        # Calculate average validation loss for the epoch
        avg_val_loss = val_loss / len(val_loader.dataset)
        
        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Check if current validation loss is the best so far
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            # Save the model state dictionary
            best_model_state = model.state_dict()
            print("Validation loss improved. Saving best model state.")
    
    if best_model_state:
        torch.save(best_model_state, './models/'+name+'_best.pth') # Save the best model
        print("Training complete. models saved to models/")
    else:
        print("Training complete. Could not save best model state.")
    

In [12]:
model = lstm_model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss()

print("\nStarting model training...")
train_model(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=40, 
    device=device,
    name='lstm_7f7d_rec'
)

lstm_model=model


Starting model training...


Epoch 1/40 Validating: 100%|██████████| 25/25 [00:00<00:00, 466.61it/s]


Epoch 1/40: Train Loss: 0.0578, Val Loss: 0.0752
Validation loss improved. Saving best model state.


Epoch 2/40 Validating: 100%|██████████| 25/25 [00:00<00:00, 461.18it/s]


Epoch 2/40: Train Loss: 0.0493, Val Loss: 0.0640
Validation loss improved. Saving best model state.


...

Epoch 38/40: Train Loss: 0.0205, Val Loss: 0.0318


Epoch 39/40 Validating: 100%|██████████| 25/25 [00:00<00:00, 631.83it/s]


Epoch 39/40: Train Loss: 0.0204, Val Loss: 0.0317


Epoch 40/40 Validating: 100%|██████████| 25/25 [00:00<00:00, 590.64it/s]

Epoch 40/40: Train Loss: 0.0204, Val Loss: 0.0319
Training complete. models saved to models/


In [13]:
model = cnn_lstm_model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss()

print("\nStarting model training...")
train_model(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=40,
    device=device,
    name='cnn_lstm_7f7d_rec'
)

cnn_lstm_model=model


Starting model training...


Epoch 1/40 Validating: 100%|██████████| 25/25 [00:00<00:00, 330.67it/s]


Epoch 1/40: Train Loss: 0.0526, Val Loss: 0.0686
Validation loss improved. Saving best model state.


Epoch 2/40 Validating: 100%|██████████| 25/25 [00:00<00:00, 306.27it/s]


Epoch 2/40: Train Loss: 0.0467, Val Loss: 0.0591
Validation loss improved. Saving best model state.


...

Epoch 38/40: Train Loss: 0.0205, Val Loss: 0.0329


Epoch 39/40 Validating: 100%|██████████| 25/25 [00:00<00:00, 315.55it/s]


Epoch 39/40: Train Loss: 0.0204, Val Loss: 0.0327


Epoch 40/40 Validating: 100%|██████████| 25/25 [00:00<00:00, 531.89it/s]


Epoch 40/40: Train Loss: 0.0204, Val Loss: 0.0332
Training complete. models saved to models/
